In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import scrublet as scr
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

# Input samples
samples = [
   "GSM6321073_PFC_4wk_1_matrix.h5",
   "GSM6321077_PFC_90wk_1_matrix.h5",
]

# Mouse mapping
mouse_id = {0: 1, 1: 1}

# Load and process samples
all_adata = []
i = 0
for s in samples:
    label, area, age, idx, _ = s.split("_")
    curr_adata = sc.read_10x_h5(f"/Users/cmdb/qb25project/mouse-brain-RNAseq/GSE207848_RAW/{s}")
    curr_adata.var_names_make_unique()

    curr_adata.obs["area"] = area
    curr_adata.obs["age"] = age
    curr_adata.obs["idx"] = i
    i += 1

    curr_adata.var["mt"] = curr_adata.var_names.str.startswith("mt-")
    sc.pp.calculate_qc_metrics(curr_adata, qc_vars=["mt"], inplace=True)

    all_adata.append(curr_adata)

# Concatenate
adata = ad.concat(all_adata)
adata.obs_names_make_unique()

# Filtering
sc.pp.filter_cells(adata, min_genes=1000)
sc.pp.filter_cells(adata, max_counts=100000)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_counts=2500)

# Mouse ID
adata.obs["mouse_id"] = [mouse_id[i] for i in adata.obs.idx]

# Scrublet doublet removal
scrub = scr.Scrublet(adata.X, expected_doublet_rate=0.09)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_gene_variability_pctl=85, n_prin_comps=30)
adata = adata[~predicted_doublets, :]

# Normalize + log
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# HVGs, PCA, neighbors, Leiden, UMAP
sc.pp.highly_variable_genes(adata)
sc.tl.pca(adata)
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=20)
sc.tl.leiden(adata, resolution=0.6)
sc.tl.umap(adata)

# Cell type scoring pipeline

marker_genes = {
   "Excitatory Neurons": ["SLC17A7", "CAMK2A", "GRIN2A"],
   "Macrophages": ["CD68", "ITGAX", "CSF1R"],
   "Oligodendrocytes": ["OLIG1", "OLIG2", "MBP"],
   "Microglia": ["CX3CR1", "ITGAX", "P2RY12"],
   "OPC": ["PDGFRA", "SOX10", "OLIG2"],
   "Peri/VLMC": ["PDGFRB", "COL1A1", "ACTA2"],
   "Astrocytes": ["GFAP", "ALDH1L1", "SLC1A3"],
   "MSN": ["DRD1", "PVALB", "SLC6A3"],
   "Inhibitory Neurons": ["GAD1", "GAD2", "SLC32A1"]
}

# Upper-case gene names
adata.var_names = [g.upper() for g in adata.var_names]

# Score gene signatures
for cell_type, genes in marker_genes.items():
    valid_genes = [g for g in genes if g in adata.var_names]
    if len(valid_genes) > 0:
        sc.tl.score_genes(adata, valid_genes, score_name=f"{cell_type}_score")

# Assign predicted cell types
score_cols = [col for col in adata.obs.columns if col.endswith("_score")]
adata.obs["predicted_cc_type"] = adata.obs[score_cols].idxmax(axis=1).str.replace("_score", "")

# Create age.tsv 
with open("/Users/cmdb/qb25project/mouse-brain-RNAseq/age.tsv", "w") as f:
    f.write("idx\tage\n")
    for i in range(len(adata.obs["age"])):
        f.write(adata.obs_names[i] + "\t" + adata.obs["age"][i] + "\n")

# Create cell_types.tsv
with open("/Users/cmdb/qb25project/mouse-brain-RNAseq/cell_types.tsv", "w") as f:
    f.write("idx\tcell_type\n")
    for i in range(len(adata.obs["predicted_cc_type"])):
        f.write(adata.obs_names[i] + "\t" + adata.obs["predicted_cc_type"][i] + "\n")

# Make .h5ad file for R 
# Ensure sparse matrix
adata.X = csr_matrix(adata.X)

# Make obs/var strings
for col in adata.obs.columns:
    adata.obs[col] = adata.obs[col].astype(str)

for col in adata.var.columns:
    if adata.var[col].dtype == "object":
        adata.var[col] = adata.var[col].astype(str)

# Remove everything that breaks Seurat
adata.uns = {}

# Save final file
adata.write("/Users/cmdb/qb25project/mouse-brain-RNAseq/adata_for_seurat.h5ad")


/opt/anaconda3/envs/qbproject/lib/python3.13/site-packages/anndata/_core/anndata.py:1798: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/opt/anaconda3/envs/qbproject/lib/python3.13/site-packages/anndata/_core/anndata.py:1798: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/opt/anaconda3/envs/qbproject/lib/python3.13/site-packages/anndata/_core/anndata.py:1798: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/opt/anaconda3/envs/qbproject/lib/python3.13/site-packages/anndata/_core/anndata.py:1798: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/opt/anaconda3/envs/qbproject/lib/python3.13/site-packages/anndata/_core/anndata.py:1796: UserWarning: Obser

Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.20
Detected doublet rate = 9.2%
Estimated detectable doublet fraction = 66.2%
Overall doublet rate:
	Expected   = 9.0%
	Estimated  = 13.9%
Elapsed time: 26.9 seconds


/opt/anaconda3/envs/qbproject/lib/python3.13/site-packages/scanpy/preprocessing/_normalization.py:269: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/anaconda3/envs/qbproject/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:245: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  Version(ad.__version__) < Version("0.9")
/opt/anaconda3/envs/qbproject/lib/python3.13/site-packages/scanpy/neighbors/__init__.py:430: FutureWarning: Use obsm (e.g. `k in adata.obsm` or `adata.obsm.keys() | {'u'}`) instead of AnnData.obsm_keys, AnnData.obsm_keys is deprecated and will be removed in the future.
  if "X_diffmap" in adata.obsm_keys():
/var/folders/m9/f9b1skd55v35_qp4hcflk2qh0000gp/T/ipykernel_1344/2632080950.py:100: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior).